# Configuration

In [1]:
import os
import json
import camelot

from tqdm import tqdm
import camelot
import numpy as np

from langchain.document_loaders import PDFMinerLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import Chroma
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
with open('../credentials.json', 'r') as f:
    credentials = json.load(f)

# Load and split documents

In [3]:
def load_document(file_path: str) -> Document:
    loader = PDFMinerLoader(file_path)
    return loader.load()[0]

In [4]:
def processing_content(content: str) -> str:
    return content.replace("  ", " ")\
                  .replace("\xad\n\n", "")\
                  .replace("\xad\n", "")\
                  .replace(" \n\n", " ")\
                  .replace(" \n", " ")\
                  .replace("\x0c", "\n")

In [5]:
data_dir = '../docs'
file_paths = os.listdir(data_dir)[:1]

In [6]:
%%time
source_documents = [load_document(os.path.join(data_dir, path)) for path in file_paths]

CPU times: total: 7.56 s
Wall time: 8.39 s


In [7]:
%%time
for doc in source_documents:
    doc.page_content = processing_content(doc.page_content)

CPU times: total: 0 ns
Wall time: 4.01 ms


In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100, add_start_index=True
)
documents = text_splitter.split_documents(source_documents)

# Load embeddings

In [9]:
OPENAI_API_KEY=credentials['OPENAI_API_KEY']

In [10]:
embeddings=OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [11]:
# class MyEmbeddingFunction(EmbeddingFunction):
#     def embed_documents(self, input: Documents) -> Embeddings:
# #         return np.random.randn(768).tolist()
#         return [1]*768
#     def embed_query(self, input: str) -> Embeddings:
# #         return np.random.randn(768).tolist()
#         return [1]*768
# embeddings=MyEmbeddingFunction()

In [12]:
vectorstore = Chroma(collection_name='engineering_store',
                     embedding_function=embeddings,
                     persist_directory="../db")

collection = vectorstore.get()

if len(collection['ids']) == 0:
    
    for doc in tqdm(documents[30:50]):
        vectorstore.add_documents([doc])

    vectorstore.persist()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.81it/s]


In [20]:
collection = vectorstore.get()

In [29]:
doc_dict = dict()
for i, metadata in enumerate(collection['metadatas']):
    doc_dict.setdefault(doc.metadata['source'][len(data_dir)+1:], list())
    doc_dict[doc.metadata['source'][len(data_dir)+1:]].extend([collection['ids'][i]])

In [30]:
doc_dict

{'Аппаратура малогабаритная электрическая. Автоматические выключатели для защиты от сверхтоков бытового и аналогичного назначения. Часть 1. Автоматические выключатели для переменного тока.pdf': ['91ff73cd-aa40-11ee-92c7-40ec99f39a7a',
  '92841984-aa40-11ee-8794-40ec99f39a7a',
  '92b12015-aa40-11ee-8aef-40ec99f39a7a',
  '92e9bc0e-aa40-11ee-a201-40ec99f39a7a',
  '931ada72-aa40-11ee-b5b2-40ec99f39a7a',
  '93490e07-aa40-11ee-b611-40ec99f39a7a',
  '9374e78e-aa40-11ee-a074-40ec99f39a7a',
  '93c6293f-aa40-11ee-8c9c-40ec99f39a7a',
  '93fc7f7b-aa40-11ee-a877-40ec99f39a7a',
  '94295989-aa40-11ee-9cbc-40ec99f39a7a',
  '94567f5d-aa40-11ee-a009-40ec99f39a7a',
  '94837bdf-aa40-11ee-acdb-40ec99f39a7a',
  '94b00ae4-aa40-11ee-9ad2-40ec99f39a7a',
  '94d9c58f-aa40-11ee-b6ae-40ec99f39a7a',
  '9506f3df-aa40-11ee-bd43-40ec99f39a7a',
  '953fdf46-aa40-11ee-ba01-40ec99f39a7a',
  '956c9c56-aa40-11ee-95ea-40ec99f39a7a',
  '95ad44d1-aa40-11ee-a81e-40ec99f39a7a',
  '95e6609a-aa40-11ee-85e3-40ec99f39a7a',
  '9610a0

In [32]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Load model

In [16]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=OPENAI_API_KEY)

# Relevant documents

In [34]:
relevant_documents = []
retrieve_documents = retriever.get_relevant_documents("Какое номинальное напряжение электроустановки для трёхфазной системы при номинальном импульсном выдерживаемом напряжении 4 В?")
for doc in retrieve_documents:
    doc_name = doc.metadata['source'][8:-4]
    if doc_name not in relevant_documents:
        relevant_documents.append(doc_name)

In [35]:
relevant_documents

['Аппаратура малогабаритная электрическая. Автоматические выключатели для защиты от сверхтоков бытового и аналогичного назначения. Часть 1. Автоматические выключатели для переменного тока']

# QnA

In [21]:
template = "Контекст: {context}\n\nИспользуя контекст, ответь на вопрос: {question}"
prompt = ChatPromptTemplate.from_template(template)

In [22]:
output_parser = StrOutputParser()

In [23]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser

In [199]:
chain.invoke("Где публикуется информация  о  введении  в  действие стандарта?")

'Информация о введении в действие стандарта публикуется в указателях национальных стандартов, издаваемых в соответствующих государствах, а также в сети Интернет на сайтах соответствующих национальных органов по стандартизации.'

In [32]:
chain.invoke("Какое номинальное напряжение электроустановки для трёхфазной системы при номинальном импульсном выдерживаемом напряжении 4 В?")

'Номинальное напряжение электроустановки для трёхфазной системы при номинальном импульсном выдерживаемом напряжении 4 В составляет 230/400 В.'